# Scaling UP: Large Language Models


Finally! Let's turn to the more recent events, the advent of Large Language Models (LLMs).

![finally](https://media.giphy.com/media/hZj44bR9FVI3K/giphy.gif)

Most the materials in this Notebook based on:
- A recent [survey article](https://arxiv.org/abs/2303.18223) on Large Language Models
- The [Stanford Course CS324](https://stanford-cs324.github.io/winter2023/assignment/) on Advances in Foundation Models

## Focus 
- Context, large, larger, largest? (Theory)
- Accessing LLMs (Practical)
- Interacting with LLMs (Practical)

## Large Language Models
- Scaling pretrained language models improves performance*
- Scaling refers to increasing model size, data and compute 
 
![model_size](https://s10251.pcdn.co/wp-content/uploads/2023/03/2023-Alan-D-Thompson-AI-Bubbles-Rev-7b.png)


*performance on tasks the ML/NLP cares about ("benchmarking")

### Scaling leads to qualitatively different (i.e. better?) models

Three differences between PLMs and LLMs (from the survey paper):
- LLMs **might** display emergent abilities that are not observed in smaller PLMs.
- LLMs would revolutionize the way we use AI algorithms: prompting, i.e. formulate a task so that LLMs can "understand" or at least follow
- "Development of LLMs no longer draws a clear distinction between research and engineering."

### LLMs are general-purpose language task solvers

- Imagine you want to automatically classify documents, by genre, emotion, topic


- PLM vs LLM: What does "Large" mean?
- Are LLMs qualitatively different than PLMs
- Different capabilities
    - Traditionally: learn from examples
        - adapt a model to a set of examples (training/fine-tuning)
    - No adaptation needed, prompting instead if traing
        - In context learning
        - Zero and few-shot 
        - Chain of thought reasoning
- "Emerging" Capabilities
    - Ideological dimensions behind the AI discourse

- Programmatic Access to LLMs

- Using LLMs: from checkpoints or via API
    - open and closed, [paper](https://www.nature.com/articles/d41586-023-01295-4)
        - risks
    - hard to say which will turn out to b


In [ ]:
# A Critique of LLMs
Stochastic Parrots.

# Checkpoint: Hugging Face and BLOOM

Introduction to BLOOM. Based on Stanford Course
https://colab.research.google.com/drive/13gyUcsX7KtkwSJ1PfW8MrlXQePVD_jFP

In [ ]:
!pip install transformers torch datasets  accelerate bitsandbytes

In [ ]:
import transformers
import torch
from datasets import load_dataset
from transformers import pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = "bigscience/bloom-1b7"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Write a zero-shot prompt
sample_review = 'I really love this movie'
prompt = f"""Classify the following movie review as positive or negative

Review: {sample_review}
Sentiment:"""

print(prompt)

In [ ]:
# Feed prompt to model to generate an output
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
output = generator(prompt, max_new_tokens=20)
print(output[0]['generated_text'])

In [ ]:
""" 
Write a few-shot prompt. Here we include a few in-context examples to the model 
demonstrating how to complete the tasks
"""

prompt = f"""Review: The movie was horrible
Sentiment: Negative

Review: The movie was the best movie I have watched all year!!!
Sentiment: Positive

Review: {sample_review}
Sentiment:"""

print(prompt)

In [ ]:
# Feed prompt to model to generate an output
output = generator(prompt, max_new_tokens=1)
print(output[0]['generated_text'])

In [ ]:
%%bash
wget https://bl.iro.bl.uk/downloads/59a8c52f-e0a5-4432-9897-0db8c067627c?locale=en -O animacy.zip 

In [ ]:
%%bash
unzip animacy.zip

In [ ]:
import pandas as pd
df = pd.read_csv('/content/LwM-nlp-animacy-annotations-machines19thC.tsv', index_col=0, sep='\t')

In [ ]:
df[df.animacy==0].head(3)

In [ ]:
df[(df.animacy==0) & (df.TargetExpression=='machine') ].head(10).Sentence.values

In [ ]:
df[df.animacy==1].head(3)

In [ ]:
df[df.animacy==1].head(3).Sentence.values

In [ ]:
target_sentence = "When the ***machine*** has been let down into the sea, and the coral is thought sufficiently"
prompt = f"""We want to know if the word ***machine*** in the following sentences is animate.
With animacy we mean the property of being alive

Sentence: Immured in a convent, debarred from life-giving air and light, and the beauty of life, we cease to be living, feeling, thinking girls and women, we become mere ***machines*** who blindly obey the head that directs us.'
Animacy: Animate

Sentence: Now that we were free from all fear of encountering bad cha racters in the house, the boom-boom of the little man's big voice went on unintermittingly, like a ***machine*** at work in the neigh bourhood
Animacy: Animate

Sentence: He led his ***machine*** to the side of thi_ footpath. 
Animacy: Inanimante

Sentence: The drawing shows the ***machine*** ready to begin its forward stroke.'
Animacy: Inanimante

Sentence: {target_sentence}
Animacy: 
"""

print(prompt)

In [ ]:
# Feed prompt to model to generate an output
output = generator(prompt, max_new_tokens=2)
print(output[0]['generated_text'])

In [ ]:
test_df = df[["Sentence",'animacy']].sample(10).replace({"animacy":{1: 'Animate',0: 'Inanimate'}})

In [ ]:
def prompt_template(target_sentence):
    return f"""We want to know if the word ***machine*** in the following sentences is animate.
    With animacy we mean the property of being alive

    Sentence: Immured in a convent, debarred from life-giving air and light, and the beauty of life, we cease to be living, feeling, thinking girls and women, we become mere ***machines*** who blindly obey the head that directs us.'
    Animacy: Animate

    Sentence: Now that we were free from all fear of encountering bad cha racters in the house, the boom-boom of the little man's big voice went on unintermittingly, like a ***machine*** at work in the neigh bourhood
    Animacy: Animate

    Sentence: He led his ***machine*** to the side of thi_ footpath. 
    Animacy: Inanimante

    Sentence: The drawing shows the ***machine*** ready to begin its forward stroke.'
    Animacy: Inanimante
    
    Sentence: {target_sentence}
    Animacy: 
    """

In [84]:
headlines = pd.read_csv('../data/emotion.csv')
headlines = headlines[headlines.apply(lambda x : len(str(x.sentence)) >= 10, axis=1)]
headlines.shape

(23528, 9)

In [85]:
top_n = 100
print('\n'.join(headlines.sort_values('neutral', ascending=False)[:top_n].sentence.values))

REPORT ON LEPROSY.
ROMAN REMAINS AT CHESTER.
_ . LATE STRIKE OR LOOK-OUT OF LONDON TAILORS.
LIGHT CAVALRY CHARGE AT BALACLA.VA.
SHIP-JOINERS LOCK-OUT ON TaL
CO-OPERATIV:
PA U.P.E RISli AND CRIME.
FOREIGN TELEGRAMS. FRANCE AND AMERICA.
VISIT TO DAHOMEY.
DENMARK AND PRUSSIA.
DENMARK AND PRUSSIA.
R.EGISTERED POR TRANSMISSION ABROAD
T INTELLIU-E N CE.
EDI/VP. GROVE,
C ii_RISTMAS ENTE RT MENTS
VISIT OF GARIBALDI.
CAPE MAIL.
CAPE MAIL.
CAPE MAIL.
VISIT TO CHISWICK.
ARBITRITION AT •HUDDERSIFIELD.
MR. ERNEST JONES ON DENIOCRA.CY.
MAYOR OF BELFAST.
"HALF CLEAR BEN EFIT P"
AUSTRIA AND PRUSSIA.
AUSTRIA AND PRUSSIA.
AUSTRIA AND PRUSSIA.
AUSTRIA AND PRUSSIA.
RUSSIA. AND AUSTRIA.
CYCLONE AT CALCUTTA.
LATER FROX
PLATE ROBBERY AT HAMPSTEAD.
LIST OF PRICES.
MR. GOLDWIN SMITH ON CANADA.
VISIT TO DR. STIORTHOUSE.
MR. BRIGHT, M.P., ON REFORM.
PRUSSIA AND DENMARK.
1.4 ICOD REFORMATION NECESS ARY.
ROYAL -UNITED SERVICE INSTITUTION.
CALCUTTA AND CHINA MAILS
LONDON ILLRKETS.
Tlig CAPE MAIL.
READ THE NEW MEDIC

# API: Accessing OpenAI's GPT-3

Full documentation is available [here](https://platform.openai.com/docs/api-reference/completions/create).

In [ ]:
# Hey ChatGPT how can I ask a question to
import openai

# Set up your OpenAI API credentials
openai.api_key = 'YOUR_API_KEY'

# Define the function to ask a question
def ask_question(question):
    prompt = f"Question: {question}\nAnswer:"

    # Generate a response from ChatGPT
    response = openai.Completion.create(
        engine='text-davinci-003', # Select the model you want to use
        prompt=prompt,  # Your query as a prompt
        max_tokens=50,  # Adjust the max tokens according to your needs
        n=1, # Number of completions to generate
        stop=None, # 
        temperature=0.7 # Regulate the LLM creativity. Lower values will produce more similar responses
    )

    # Extract and return the answer from the response
    answer = response.choices[0].text.strip().split('\n')[0]
    return answer

# Ask a question to ChatGPT
question = "What is the capital of France?"
answer = ask_question(question)
print(answer)

In [33]:
!pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 8.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for lxml: filename=lxml-4.9.2-cp39-cp39-macosx_11_0_arm64.whl size=1567555 sha256=538f09138c1a799b3780432b7eb024d42ede7914498fa1c8278762f01d3eabb7
  Stored in directory: /Users/kasparbeelen/Library/Caches/pip/wheels/74/7c/5a/e117656a962a1a15a3d2ac1bde4bc6193d62dc5d7e9c51e15e
Successfully built lxml


In [62]:
# Prep data

from pathlib import Path
xml_files = Path('../data/0002247').glob('**/*.xml')

In [63]:
from lxml import etree
def get_title(path):
    try:
        with open(path,'rb') as xml:
            tree = etree.parse(xml)
        return tree.xpath('//item/title')[0].text
    except:
        return ''

In [64]:
titles = list(map(get_title,xml_files))

In [65]:
titles[:10]

['NOTICE TO CORRESPONDENTS,',
 'POLICE INTELLIGENCE.',
 'NAVAL AND MILITARY.',
 'WANDSWORTH.',
 'BRITISH MUSEUM',
 "PROCLAMATION OF THE DANISH COMMANDER4N;C'HIEF. '",
 'DEATH FROM DESTITUTION.',
 None,
 'MEXICO.',
 'FOREIGN TELEGRAMS.']

In [66]:
titles = [t for t in titles if t]

In [68]:
text = '\n'.join([t.strip() for t in titles])
with open('../data/0002247_titles.txt','w') as out_text:
    out_text.write(text)
    